# Task 
requirements
yfinance
numpy
pandas


In [ ]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px
 
import time
from datetime import date


In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
df = pd.read_html(url)[0]
df.loc[:, "Date added"] = pd.to_datetime(df["Date added"])
df.loc[:, "year"] = pd.to_datetime(df["Date added"]).dt.year
group = df.groupby("year").agg({
    "Symbol": lambda x: x.notna().sum()
})
group.sort_values("Symbol", ascending=False)
group.head()

In [ ]:
df.loc[:, "year since"] = date.today().year -  pd.to_datetime(df["Date added"]).dt.year
filtered_df = df.loc[(df["year since"] > 20) & (df["year"] > 1957)].sort_values("year since", ascending=False)
print("How many companies are in the index for more than 20 year", len(filtered_df))
filtered_df.reset_index(inplace=True)
filtered_df

In [ ]:
# Load table
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
data = pd.read_html(url, header=[0, 1])[1]

# Safely assign datetime to the original DataFrame
date_col = pd.to_datetime(data[('Date', 'Date')])
data.loc[:, ('Date', 'Date')] = date_col

# Now extract year and compute year difference
data.loc[:, ('Date', 'year')] = date_col.dt.year
data.loc[:, ('Date', 'year since')] = date.today().year - date_col.dt.year

# Check result
filtered = data.loc[data[('Date', 'year since')] > 20]
filtered.sort_values(('Date', 'year since'), ascending=False)
print("how may companies are in the index more than 20 years: ", len(filtered))
filtered

# Question 2


In [ ]:
import datetime
end = date(2025, 5, 1) # As of May 1
start = date(2025, 1,1 )
print(f'Year = {end.year}; month= {end.month}; day={end.day}')
print(f'Period for indexes: {start} to {end} ')

def get_ytd_value(ticket:str, start:datetime.date=date(2025, 1, 1), end:datetime.date=date(2025,5 ,1 )):
    """return the YTD from a ticket from yahoo finance in %

    """

    df = yf.download(
        tickers = ticket,
        start=start, 
        end=end,
        interval = "1d")
    ytd = (df[( 'Close', ticket)].iloc[-1]/df[( 'Close', ticket)].iloc[0] -1)*100
    return float(ytd)


In [ ]:
countries = {
    "USA": "^GSPC",                         # S&P 500
    "China": "000001.SS",                   # Shanghai Composite
    "Hong Kong": "^HSI",                    # Hang Seng Index
    "Australia": "^AXJO",                   # S&P/ASX 200
    "India": "^NSEI",                       # Nifty 50
    "Canada": "^GSPTSE",                    # S&P/TSX Composite
    "Germany": "^GDAXI",                    # DAX
    "United Kingdom": "^FTSE",              # FTSE 100
    "Japan": "^N225",                       # Nikkei 225
    "Mexico": "^MXX",                       # IPC Mexico
    "Brazil": "^BVSP"                       # Ibovespa
}

data = []

for country, ticket in countries.items():
    ytd = get_ytd_value(ticket)
    data.append({
        "Country": country,
        "Ticket": ticket,
        "YTD": ytd
    })

df = pd.DataFrame(data)

# Get USA YTD for comparison
usa_ytd = df.loc[df["Country"] == "USA", "YTD"].values[0]

# Calculate difference from USA
df["S&P Diff"] = df["YTD"] - usa_ytd

# Optional: sort by difference
df = df.sort_values("S&P Diff", ascending=False)
df.reset_index(inplace=True)
del df["index"]

In [ ]:
df

# Question 4


    Download S&P 500 historical data (1950-present) using yfinance
    Identify all-time high points (where price exceeds all previous prices)
    For each pair of consecutive all-time highs, find the minimum price in between
    Calculate drawdown percentages: (high - low) / high × 100
    Filter for corrections with at least 5% drawdown
    Calculate the duration in days for each correction period
    Determine the 25th, 50th (median), and 75th percentiles for correction durations


In [ ]:
df = yf.download(
    tickers = "^GSPC",
    start=date(1950, 1, 1), 
    end=date.today(),
    interval = "1d")["Close"]
df.loc[:, "cummax"] = df["^GSPC"].cummax()

In [ ]:
# Prepare an empty DataFrame to collect corrections with >5% drawdown
corrections_df = pd.DataFrame()

for i in df["cummax"].unique():
    filtered_df = df[df["cummax"] == i].copy()

    filtered_df["diff"] = np.round((1 - filtered_df["^GSPC"] / filtered_df["cummax"]) * 100, 1)

    low_row = filtered_df.loc[filtered_df["^GSPC"].idxmin()] 
    min_price = low_row["^GSPC"]

    high_prices = filtered_df["cummax"]
    filtered_df["drawdown"] = np.round((high_prices - min_price) / high_prices * 100, 1)

    # Assuming filtered_df.index is a DatetimeIndex
    filtered_df['days_from_start'] = (filtered_df.index - filtered_df.index[0]).days
    date_started = filtered_df.index[0]


    # Get the first date where drawdown equals diff
    df_to_append = filtered_df[filtered_df["drawdown"] == filtered_df["diff"]].copy()
    df_to_append.loc[:, "Date_started"] = date_started
    df_to_append.index.rename("Date_ended", inplace=True)

    if df_to_append["drawdown"].iloc[0] > 5:
        corrections_df = pd.concat([corrections_df, df_to_append])

# Optional: reset index after concatenation
corrections_df = corrections_df.reset_index()
corrections_df  = corrections_df[[ 'Date_started', 'Date_ended', 'drawdown', 'days_from_start']]


In [ ]:
corrections_df[corrections_df["Date_started"] == pd.Timestamp("1966-02-09")]

In [ ]:
corrections_df["days_from_start"].median()

# Question 4


In [ ]:
url = "https://raw.githubusercontent.com/DataTalksClub/stock-markets-analytics-zoomcamp/refs/heads/main/cohorts/2025/ha1_Amazon.csv"
earnings_df = pd.read_csv(url, delimiter=';')
# Convert date column to datetime
# Convert 'Earnings Date' to datetime
# Strip time zone info if parsing fails
earnings_df['Earnings Date'] = earnings_df['Earnings Date'].str.replace(r' [A-Z]{3}$', '', regex=True)
earnings_df['Earnings Date'] = pd.to_datetime(earnings_df['Earnings Date'], format="%B %d, %Y at %I %p")


import re

def clean_eps(value):
    if pd.isna(value):
        return None
    # Remove non-numeric characters except period and minus sign
    cleaned = re.sub(r"[^0-9\.\-]", "", str(value))
    try:
        return float(cleaned)
    except ValueError:
        return None



# Rename for simplicity
earnings_df.rename(columns={
    'Earnings Date': 'date',
    'EPS Estimate': 'estimate',
    'Reported EPS': 'actual'
}, inplace=True)

earnings_df['estimate'] = earnings_df['estimate'].apply(clean_eps)
earnings_df['actual'] = earnings_df['actual'].apply(clean_eps)
earnings_df = earnings_df.dropna(subset=['estimate', 'actual'])


[*********************100%***********************]  1 of 1 completed


In [284]:
amazon = yf.download(
    tickers = "AMZN", 
    end=date.today(),
    interval = "1d")["Close"]
amazon['Close_Day3'] = amazon['AMZN'].shift(2)
amazon['Close_Day1'] = amazon['AMZN']
amazon['2d_return'] = amazon['Close_Day3'] / amazon['Close_Day1'] - 1
amazon

[*********************100%***********************]  1 of 1 completed


Ticker,AMZN,Close_Day3,Close_Day1,2d_return
Date,,,,
1997-05-15,0.097917,NaN,0.097917,NaN
1997-05-16,0.086458,NaN,0.086458,NaN
1997-05-19,0.085417,0.097917,0.085417,0.146341
1997-05-20,0.081771,0.086458,0.081771,0.057319
1997-05-21,0.071354,0.085417,0.071354,0.197088
...,...,...,...,...
2025-05-28,204.720001,200.990005,204.720001,-0.018220
2025-05-29,205.699997,206.020004,205.699997,0.001556
2025-05-30,205.009995,204.720001,205.009995,-0.001415


In [285]:
earnings_df = earnings_df[earnings_df['actual'] > earnings_df['estimate']].copy()
earnings_df = earnings_df.sort_values('date').reset_index(drop=True)

print(f"Number of positive surprise dates: {len(earnings_df)}")

Number of positive surprise dates: 36


In [286]:
# Remove timezone from earnings dates (if present)
earnings_df['date'] = earnings_df['date'].dt.tz_localize(None)

# Ensure index is datetime and timezone-naive
amazon.reset_index(inplace=True)
amazon['Date'] = pd.to_datetime(amazon['Date'])
amazon = amazon.set_index('Date')
amazon.index = amazon.index.tz_localize(None)

# Define the lookup function
def find_next_trading_day(earn_date):
    return amazon.index[amazon.index >= earn_date].min()

# Apply to earnings dates
earnings_df['trading_date'] = earnings_df['date'].apply(find_next_trading_day)


In [287]:
# Merge to get 2-day return for earnings dates

merged = pd.merge(
    earnings_df,
    amazon[['2d_return']],
    left_on='trading_date',
    right_index=True,
    how='left'
)

median_positive_surprise_return = merged['2d_return'].median() * 100
print(f"Median 2-day return after positive surprises: {median_positive_surprise_return:.2f}%")


Median 2-day return after positive surprises: -2.77%


In [288]:
median_all_returns = amazon['2d_return'].median() * 100
print(f"Median 2-day return for all days: {median_all_returns:.2f}%")


Median 2-day return for all days: -0.17%
